# Exploring cursors

For each code cell, try to **predict** the output before you run it!  Discuss with a partner.

## Creating a search cursor

The cursor can be created with just the full path file name of the data and a list of field names.

In [ ]:
import arcpy
fc = "C:/gispy/data/ch17/park.shp"  
fds = arcpy.ListFields(fc)
fds

How many fields does the data have?

In [ ]:
len(fds)

(Q1) Why does following code throw an exception?  

In [ ]:
sc = arcpy.da.SearchCursor(fc, fds)
#(Hint:  What does SearchCursor need for the 2nd argument?)

(Q2) How can you get a list of the field names (with no hard-coding)?

In [ ]:
# Replace hard-coded list in line 4 so that the
# code will still work if a field is removed. 
# (Hint: you can derive it from fds)
field_names = ['FID', 'Shape', 'COVER', 'RECNO']
field_names

Let's try to create the SearchCursor again!

In [ ]:
sc = arcpy.da.SearchCursor(fc, field_names)
sc

## Traversing rows
The rows can be traversed by calling *next()* or using a FOR-loop (or a combination of these).

Here we get the first row of data by calling *next()* on the SearchCursor object.  

In [ ]:
row = sc.next()
print(f"row = {row}")
print(f"row[0] = {row[0]}")

What piece of data does *row[0]* represent?  What is the value of *row[2]*?

In [ ]:
row[2]

This is a polygon shapefile.  Notice the 2nd field, the shape field, it prints the coordinates of the polygon's centroid.  We'll revisit the geometric features in a moment.

In [ ]:
# Print the shape field
row[1]

Let's get the next row of data.  What will *row[0]* be?

In [ ]:
row = sc.next()
row[0]

What does the cursor reset method do?  Run the next cell and observe the data to see if you guessed correctly.

In [ ]:
sc.reset( )

row = sc.next()
print(f"row a = {row}")

row = sc.next()
print(f"row b = {row}")

*next()* is good for exploration, but more often you'll want to run through all the rows.  The cursor is an iterator, so you can just pop it in a FOR-loop. 

Note that it starts with the **3rd row** (where the *next()* left off) because that's where we left the cursor. 

In [ ]:
for row in sc:
    print(row)

(Q3) Why does the following code throw an error?

In [ ]:
del sc
sc

## Working with geometry

When we created our cursor with the fields 'FID', 'Shape', 'COVER', 'RECNO', and 'AREA',  the 'Shape' field only provided the polygon's centroid.  By instead specifying SHAPE@ as the field name, we can get the geometry object for the record.  Geometry objects can be type Point, Multipoint, Polyline, or Polygon. 

In [ ]:
field_names = ["SHAPE@"]

sc = arcpy.da.SearchCursor(fc, field_names)
row = sc.next()
row

What data type is row?

In [ ]:
type(row)

How long many items are in row?

In [ ]:
len(row)

That's a little quirk of Python tuples.  

Tuples are created by the comma not the parentheses.

So even if there's only one item in the tuple, it will be followed by a comma when you print it.

E.g., here's a tuple with one item: ('foo',)

In [ ]:
# As you can see, trying to access a second item in the row tuple throws an exception.
row[1]

Let's give it a more meaningful name.  

In [ ]:
the_shape = row[0]

What type of object is this?

In [ ]:
type(the_shape)

If you're using this notebook inside ArcGIS Pro,  the above code may throw an error.   You can ignore that.

It's an arcpy Polygon type geometry object.  Here's how Python says the same thing:  

arcpy.arcobjects.geometries.Polygon

What type of shape is this?

In [ ]:
the_shape.type

What attributes do the Polygon objects have?

In [ ]:
dir(the_shape)

But how does this polygon look?

In [ ]:
the_shape

Cool!  (Q4) How about the next polygon?

In [ ]:
# Insert code to get the next row and display the polygon in that row.

### Polygon property examples
Each type of geometry object has a set of geospatial properties and methods.  Try the code below to see some of the Polygon properties.  For the complete list of Polygon properties/methods, search online for: polygon arcpy

In [ ]:
the_shape.extent

In [ ]:
the_shape.extent.XMin

(Q5) How can you find the northernmost extent of this polygon?

In [ ]:
# Insert code to find the northermost extent of this polygon.

In [ ]:
the_shape.centroid

In [ ]:
type(the_shape.centroid)

In [ ]:
the_shape.centroid.X

In [ ]:
the_shape.pointCount

In [ ]:
print(the_shape.firstPoint)
print(the_shape.lastPoint)

How can we traverse all of the polygon's points?   

In [ ]:
for part in the_shape:
    for pnt_num, pnt in enumerate(part):
        print(f"Point: {pnt_num}: {pnt.X}, {pnt.Y}")

The traversal code above assumes it's a single-part polygon and with no holes.    For the slightly more complex but more general version, search online for: arcpy reading polyline or polygon geometries

(Q6) Can you guess how to determine the area?

In [ ]:
# Insert code to get the area of the_shape.

Only the numerical value of the area is given.  What are the units for area? It will be determined by the dataset's units.   You can find this by looking at the spatial reference metadata:

In [ ]:
the_shape.spatialReference

### Polygons method examples
Next, explore some of the Polygon methods.

In [ ]:
the_shape.boundary()

To get the length, you need to specify measurement_type (e.g., PLANAR or GEODESIC) and units (FEET or METERS) 

In [ ]:
# getLength ({measurement_type}, {units})
the_shape.getLength('PLANAR', 'METERS')

In [ ]:
the_shape.convexHull()

In [ ]:
buff_polygon = the_shape.buffer(50)
buff_polygon

In [ ]:
buff_polygon.contains(the_shape) 

(Q7) Use the *difference* method to make a polygon with only the buffered zone and minus the original polygon.

In [ ]:
# The difference method constructs the geometry that is 
# composed only of the region unique to the base geometry 
# but not part of the other geometry. 
# 
# Hint: Fill in the blanks below.
#
# _______.difference(______) 

## Filtering records
The *where_clause* parameter enables you to filter the rows returned by the cursor.  This can improve performance and simplify code.

So far, we have only used the required parameters for the SearchCursor.  ArcGIS help shows the optional parameters in curly braces {}:

*arcpy.da.SearchCursor (in_table, field_names, {where_clause}, {spatial_reference}, {explode_to_points}, {sql_clause}, {datum_transformation})*

The first optional parameter, *where_clause*, enables you to filter the rows returned by the cursor.  This can improve performance and simplify code.

Here's an example that uses a where_clause.  Can you predict what it's going to do?

In [ ]:
sc = arcpy.da.SearchCursor(in_table=fc, 
                          field_names=["FID", "COVER"] , 
                          where_clause="COVER = 'other'")

# Advance the cursor to the first row it selected.
row = sc.next( )

row


How is <> interpreted in the code below?

In [ ]:
sc = arcpy.da.SearchCursor(in_table=fc, 
                          field_names=["FID", "COVER"], 
                          where_clause="COVER <> 'woods'")
for row in sc:
    print(f"row = {row}")

Predict the output of the following code.

In [ ]:
sc =arcpy.da.SearchCursor(in_table=fc, 
                          field_names=["FID", "COVER"] , 
                          where_clause="FID > 200")
row = sc.next()

print(f"row = {row}")

You can use comparison operators AND, OR, and NOT in the queries.  The example below returns only rows that have FID between 10 and 15 that have a cover type that is not orch (for orchard).

In [ ]:
sc = arcpy.da.SearchCursor(in_table=fc, 
                          field_names=["RECNO", "COVER"], 
                          where_clause="10<FID AND FID<15 AND COVER <> 'orch'")
for row in sc:
    print(f"row = {row}")

You could also use NOT to enforce inequality.

In [ ]:
sc = arcpy.da.SearchCursor(in_table=fc, 
                          field_names=["RECNO", "COVER"], 
                          where_clause="10<FID AND FID<15 AND NOT COVER = 'orch'")
for row in sc:
    print(f"row = {row}")

Chaining inequality operators is not allowed in this context.  For example, the where_clause below is considered an invalid SQL statement.

In [ ]:
sc = arcpy.da.SearchCursor(in_table=fc, 
                          field_names=["RECNO", "COVER"], 
                          where_clause="10<FID<15")
for row in sc:
    print(f"row = {row}")

(Q8) Get a cursor with only rows that have a cover type other than 'woods' and a RECNO less than 10 or greater than or equal to 420.  Hint: Consider using parentheses inside the statement to group the RECNO selection.

For more about where_clause values, search online for:  SQL reference for query expressions used in ArcGIS

## Answers

Here are responses to the questions that were not answered earlier in the notebook.

(Q1) Instantiating the SearchCursor with *fds* (as shown below) throws an error because *fds* is a list of Field objects.  
*fds = arcpy.ListFields(fc)*
But the search cursor needs a list of (string) names of fields.

In [ ]:
sc = arcpy.da.SearchCursor(fc, fds)  # DON'T DO THIS

(Q2) Still wondering how to dynamically get the field_names list?

In [ ]:
field_names = [f.name for f in fds]

You could then select a subset if desired, 
E.g., if you know you only want to use the last 2 fields:

In [ ]:
field_names = [f.name for f in fds]
field_names = field_names[-2:]

In fact, if you want to get all of the fields, you don't need the list of names.  Instead, you can use a wildcard as a placeholder for this required argument:

In [ ]:
field_names = ["*"]

(Q3) The following code throws an error because it destroys the cursor object and then tries the use the object it just destroyed.

In [ ]:
del sc
sc

(Q4) To see the next polygon...

In [ ]:
# Code to get the next row
# and display the polygon in that row. 
row = sc.next()
row[0] 


(Q5) How can you find the northernmost extent of this polygon?

In [ ]:
the_shape.extent.YMax

(Q6) How can you find the area of the_shape?

In [ ]:
# Insert code to get the area of the_shape.
the_shape.area

(Q7) Using the difference method:

In [ ]:
buff_polygon.difference(the_shape) 

(Q8) Get a cursor with only rows that have a cover type other than 'woods' and a RECNO less than 10 or greater than or equal to 420.  Hint: Consider using parentheses inside the statement to group the RECNO selection.

In [ ]:
sc = arcpy.da.SearchCursor(in_table=fc, 
                          field_names=["FID", "COVER"], 
                          where_clause="NOT COVER = 'woods' AND (RECNO>=420 OR RECNO<10)")
for row in sc:
    print(f"row = {row}")